In [4]:
#Importar bibliotecas
import pandas as pd
import numpy as np
import psycopg2 as pg2

In [ ]:
#Lendo arquivos CSV
hits=pd.read_csv('C:/Users/andre/OneDrive/Área de Trabalho/DP6/hits.csv')
products=pd.read_csv('C:/Users/andre/OneDrive/Área de Trabalho/DP6/products.csv')
sessions=pd.read_csv('C:/Users/andre/OneDrive/Área de Trabalho/DP6/sessions.csv')

In [ ]:
#Remodelar tabelas, trazer somente informações que serão usadas
products=products[['fullVisitorId','visitId','hitNumber','productPrice','productQuantity','productRevenue','v2ProductCategory','v2ProductName']]
sessions=sessions[['fullVisitorId','visitId','date','visitNumber','salesRegion','channelGrouping','geoNetwork_country','geoNetwork_continent']]
hits=hits[['hitNumber','fullVisitorId','visitId','transaction_transactionRevenue']]
hits.rename(columns={'transaction_transactionRevenue':'Revenue'},inplace=True)
#Considerar somente vendas que se concretizaram
products=products[(products['productQuantity']>0) & (products['productRevenue']>0)]
hits=hits[hits['Revenue']>0]